In [13]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, classification_report

In [14]:
device = "cuda" if torch.cuda.is_available() else "cpu"
torch.manual_seed(42)
np.random.seed(42)

In [15]:
df = pd.read_csv("/content/drive/MyDrive/[Projects]/AI Hub/Pose Detection 기반 실시간 낙상 감지 시스템 개발/Data/data_resized/낙상사고 위험동작 영상-센서 쌍 데이터_병원,후면낙상/3.개방데이터/1.데이터/Training/fall_window_dataset.csv")

meta_cols  = ['scene_id', 'label']
video_cols = ['vid_mean_flow', 'vid_max_flow', 'vid_mean_diff']
frame_cols = [c for c in df.columns if 'sk' in c or 'bbox' in c]

T = 10
F = len(frame_cols) // T

print("T:", T, "F:", F)

T: 10 F: 57


In [16]:
scaler = StandardScaler()
x_frame = scaler.fit_transform(df[frame_cols].values.astype(np.float32))
x_frame = x_frame.reshape(-1, T, F)

# video feature는 학습 안할 예정 (xv 무시)
y = df['label'].values.astype(np.float32)

In [17]:
xf_tr, xf_va, y_tr, y_va = train_test_split(
    x_frame, y,
    test_size=0.2,
    stratify=y,
    random_state=42
)

In [18]:
class FallDataset(Dataset):
    def __init__(self, xf, y):
        self.xf = torch.tensor(xf)
        self.y  = torch.tensor(y)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, i):
        return self.xf[i], self.y[i]

In [19]:
train_loader = DataLoader(FallDataset(xf_tr, y_tr), batch_size=64, shuffle=True)
val_loader   = DataLoader(FallDataset(xf_va, y_va), batch_size=64, shuffle=False)

In [20]:
class FallLSTM(nn.Module):
    def __init__(self, F, hidden=128):
        super().__init__()
        self.lstm = nn.LSTM(F, hidden, batch_first=True)
        self.fc = nn.Linear(hidden, 1)

    def forward(self, x_seq):
        _, (h, _) = self.lstm(x_seq)
        h = h[-1]
        return self.fc(h).squeeze(1)

In [21]:
model = FallLSTM(F).to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [22]:
for epoch in range(20):
    model.train()
    total_loss = 0

    for xf, yb in train_loader:
        xf, yb = xf.to(device), yb.to(device)

        optimizer.zero_grad()
        logits = model(xf)
        loss = criterion(logits, yb)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    # validation
    model.eval()
    probs, preds, gts = [], [], []
    with torch.no_grad():
        for xf, yb in val_loader:
            xf = xf.to(device)
            logit = model(xf)
            prob = torch.sigmoid(logit).cpu().numpy()

            probs.extend(prob)
            preds.extend(prob > 0.5)
            gts.extend(yb.numpy())

    f1 = f1_score(gts, preds)
    print(f"[{epoch+1:02d}] loss={total_loss:.3f} val_f1={f1:.3f}")

[01] loss=19.529 val_f1=0.923
[02] loss=7.880 val_f1=0.964
[03] loss=5.304 val_f1=0.968
[04] loss=4.435 val_f1=0.969
[05] loss=3.390 val_f1=0.965
[06] loss=2.824 val_f1=0.972
[07] loss=2.365 val_f1=0.978
[08] loss=2.024 val_f1=0.975
[09] loss=1.476 val_f1=0.975
[10] loss=1.110 val_f1=0.975
[11] loss=0.919 val_f1=0.974
[12] loss=0.548 val_f1=0.975
[13] loss=0.421 val_f1=0.972
[14] loss=0.279 val_f1=0.974
[15] loss=0.207 val_f1=0.974
[16] loss=0.153 val_f1=0.975
[17] loss=0.110 val_f1=0.974
[18] loss=0.084 val_f1=0.972
[19] loss=0.065 val_f1=0.972
[20] loss=0.053 val_f1=0.974


In [23]:
print(classification_report(gts, preds, digits=4))

              precision    recall  f1-score   support

         0.0     0.9778    0.9821    0.9800       448
         1.0     0.9765    0.9708    0.9737       343

    accuracy                         0.9772       791
   macro avg     0.9772    0.9765    0.9768       791
weighted avg     0.9772    0.9772    0.9772       791



In [24]:
MODEL_PATH = "/content/drive/MyDrive/[Projects]/AI Hub/Pose Detection 기반 실시간 낙상 감지 시스템 개발/fall_lstm.pt"

torch.save({
    "model_state": model.state_dict(),
    "input_dim": F,
    "T": T,
    "scaler": scaler
}, MODEL_PATH)

print("✅ LSTM model saved")

✅ LSTM model saved
